# Análisis de Cartola Débito Directo

## Período: 08 de Agosto al 14 de Septiembre de 2022

Este notebook realiza un análisis completo de la cartola de la cuenta de Débito Directo de Fintoc para el cierre contable del período indicado.

### Contexto del Producto

- Los usuarios suscriben un PAC para cargos recurrentes
- Los bancos debitan de las cuentas de usuarios
- Los bancos transfieren a Fintoc una suma agregada (día N)
- Fintoc concilia y paga a comercios al día hábil siguiente (N+1)

## 1. Configuración e Importación de Datos

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: f'${x:,.0f}' if abs(x) > 1 else f'${x:.2f}')

print("Librerías cargadas correctamente")

In [ ]:
# Cargar datos
df = pd.read_csv('inputs/debito_directo.csv')

# Convertir fecha a datetime
df['date'] = pd.to_datetime(df['date'])

# Mostrar primeras filas
print(f"Total de registros: {len(df)}")
print(f"Período: {df['date'].min().strftime('%Y-%m-%d')} al {df['date'].max().strftime('%Y-%m-%d')}")
print("\nPrimeros 5 registros:")
df.head()

In [ ]:
# Información del dataset
print("Estructura del dataset:")
print(df.info())
print("\nEstadísticas descriptivas:")
df.describe()

## 2. Clasificación de Movimientos

In [ ]:
# Clasificar tipo de movimiento
df['tipo_movimiento'] = df['description'].apply(
    lambda x: 'INGRESO_PAC' if 'Ingreso por PAC' in x else 'LIQUIDACION'
)

# Extraer banco (para ingresos)
df['banco'] = df['description'].apply(
    lambda x: x.replace('Ingreso por PAC Multibanco ', '').replace('Ingreso por PAC Multibco.', '')
    if 'Ingreso por PAC' in x else None
)

# Extraer comercio (para liquidaciones)
df['comercio'] = df['description'].apply(
    lambda x: x.replace('Liquidacion a: ', '') if 'Liquidacion a:' in x else None
)

# Mostrar distribución
print("Distribución de movimientos:")
print(df['tipo_movimiento'].value_counts())

## 3. Resumen General

In [ ]:
# Separar ingresos y liquidaciones
ingresos = df[df['tipo_movimiento'] == 'INGRESO_PAC'].copy()
liquidaciones = df[df['tipo_movimiento'] == 'LIQUIDACION'].copy()

# Calcular totales
total_ingresos = ingresos['amount'].sum()
total_liquidaciones = abs(liquidaciones['amount'].sum())
diferencia = total_ingresos - total_liquidaciones

# Mostrar resumen
print("=" * 60)
print("RESUMEN FINANCIERO")
print("=" * 60)
print(f"\nTotal Ingresos PAC:     ${total_ingresos:>15,.0f} CLP ({len(ingresos)} movimientos)")
print(f"Total Liquidaciones:    ${total_liquidaciones:>15,.0f} CLP ({len(liquidaciones)} movimientos)")
print("-" * 60)
print(f"DIFERENCIA:             ${diferencia:>15,.0f} CLP")
print("=" * 60)

if diferencia < 0:
    print(f"\n⚠️  ALERTA: Existe un DESCUADRE de ${abs(diferencia):,.0f} CLP")
    print("   Las liquidaciones superan a los ingresos.")

## 4. Análisis por Banco

In [ ]:
# Agrupar por banco
por_banco = ingresos.groupby('banco').agg(
    Total=('amount', 'sum'),
    Cantidad=('amount', 'count'),
    Promedio=('amount', 'mean')
).sort_values('Total', ascending=False)

# Agregar porcentaje
por_banco['% del Total'] = (por_banco['Total'] / por_banco['Total'].sum() * 100).round(1)

print("INGRESOS POR BANCO")
print("=" * 80)
print(por_banco.to_string())
print("\n" + "-" * 80)
print(f"TOTAL: ${por_banco['Total'].sum():,.0f} CLP en {por_banco['Cantidad'].sum()} movimientos")

In [ ]:
# Visualización de ingresos por banco (solo si matplotlib está disponible)
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gráfico de barras
    por_banco['Total'].plot(kind='barh', ax=axes[0], color='steelblue')
    axes[0].set_xlabel('Monto (CLP)')
    axes[0].set_title('Ingresos PAC por Banco')
    axes[0].xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
    
    # Gráfico de pie
    por_banco['Total'].plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
    axes[1].set_ylabel('')
    axes[1].set_title('Distribución de Ingresos')
    
    plt.tight_layout()
    plt.show()
except ImportError:
    print("Matplotlib no está instalado. Instálalo con: pip install matplotlib")

## 5. Análisis por Comercio

In [ ]:
# Agrupar por comercio
por_comercio = liquidaciones.groupby('comercio').agg(
    Total=('amount', lambda x: abs(x.sum())),
    Cantidad=('amount', 'count'),
    Promedio=('amount', lambda x: abs(x.mean()))
).sort_values('Total', ascending=False)

# Agregar porcentaje
por_comercio['% del Total'] = (por_comercio['Total'] / por_comercio['Total'].sum() * 100).round(1)

print("LIQUIDACIONES POR COMERCIO")
print("=" * 80)
print(por_comercio.to_string())
print("\n" + "-" * 80)
print(f"TOTAL: ${por_comercio['Total'].sum():,.0f} CLP en {por_comercio['Cantidad'].sum()} movimientos")

In [ ]:
# Visualización de liquidaciones por comercio
try:
    import matplotlib.pyplot as plt
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Top 10 comercios
    top_comercios = por_comercio.head(10)
    top_comercios['Total'].plot(kind='barh', ax=ax, color='coral')
    ax.set_xlabel('Monto (CLP)')
    ax.set_title('Top 10 Comercios por Liquidaciones')
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
    
    plt.tight_layout()
    plt.show()
except ImportError:
    print("Matplotlib no está instalado.")

## 6. Verificación del Flujo del Producto

In [ ]:
def siguiente_dia_habil(fecha):
    """Calcula el siguiente día hábil (excluye sábado y domingo)"""
    siguiente = fecha + timedelta(days=1)
    while siguiente.weekday() >= 5:  # 5=sábado, 6=domingo
        siguiente += timedelta(days=1)
    return siguiente

# Agrupar por fecha
ingresos_por_fecha = ingresos.groupby('date')['amount'].sum().to_dict()
liquidaciones_por_fecha = liquidaciones.groupby('date')['amount'].sum().abs().to_dict()

# Verificar flujo
verificacion = []
for fecha_ingreso, monto_ingreso in sorted(ingresos_por_fecha.items()):
    fecha_liq_esperada = siguiente_dia_habil(fecha_ingreso)
    monto_liquidado = liquidaciones_por_fecha.get(fecha_liq_esperada, 0)
    
    estado = 'OK' if monto_liquidado > 0 else 'SIN LIQUIDACIÓN'
    diferencia_dia = monto_ingreso - monto_liquidado if monto_liquidado > 0 else monto_ingreso
    
    verificacion.append({
        'Fecha Ingreso': fecha_ingreso.strftime('%Y-%m-%d'),
        'Día': fecha_ingreso.strftime('%A'),
        'Monto Ingreso': monto_ingreso,
        'Fecha Liq. Esperada': fecha_liq_esperada.strftime('%Y-%m-%d'),
        'Monto Liquidado': monto_liquidado,
        'Diferencia': diferencia_dia,
        'Estado': estado
    })

df_verificacion = pd.DataFrame(verificacion)

print("VERIFICACIÓN DEL FLUJO: Ingreso (Día N) → Liquidación (Día N+1 Hábil)")
print("=" * 100)
print(df_verificacion.to_string(index=False))

## 7. Detección de Anomalías

In [ ]:
print("=" * 80)
print("ANOMALÍA #1: MOVIMIENTOS CON MONTOS MUY PEQUEÑOS (<$50)")
print("=" * 80)

montos_pequenos = df[df['amount'].abs() < 50].copy()
montos_pequenos['Monto Absoluto'] = montos_pequenos['amount'].abs()

print(f"\nSe encontraron {len(montos_pequenos)} movimientos con montos < $50:\n")
print(montos_pequenos[['date', 'description', 'amount']].to_string(index=False))

In [ ]:
print("=" * 80)
print("ANOMALÍA #2: COMERCIOS CON NOMBRES DUPLICADOS")
print("=" * 80)

# Buscar duplicados por case-insensitive
comercios = liquidaciones['comercio'].unique()
comercios_lower = {}

for c in comercios:
    if c is not None:
        key = c.lower()
        if key in comercios_lower:
            comercios_lower[key].append(c)
        else:
            comercios_lower[key] = [c]

print("\nComercios con variaciones de nombre:")
for key, valores in comercios_lower.items():
    if len(valores) > 1:
        print(f"\n  Duplicados encontrados: {valores}")
        for v in valores:
            total = abs(liquidaciones[liquidaciones['comercio'] == v]['amount'].sum())
            count = len(liquidaciones[liquidaciones['comercio'] == v])
            print(f"    - '{v}': ${total:,.0f} en {count} movimientos")

In [ ]:
print("=" * 80)
print("ANOMALÍA #3: ANÁLISIS DEL DESCUADRE")
print("=" * 80)

# Analizar el 30-31 de agosto
print("\nAnálisis 30-31 de Agosto:")
print("-" * 40)

ing_30 = df[(df['date'] == '2022-08-30') & (df['tipo_movimiento'] == 'INGRESO_PAC')]['amount'].sum()
liq_31 = abs(df[(df['date'] == '2022-08-31') & (df['tipo_movimiento'] == 'LIQUIDACION')]['amount'].sum())

print(f"Ingreso 30/08: ${ing_30:,.0f}")
print(f"Liquidaciones 31/08: ${liq_31:,.0f}")
print(f"Diferencia: ${ing_30 - liq_31:,.0f}")

print("\nDetalle liquidaciones 31/08:")
liq_31_df = df[(df['date'] == '2022-08-31') & (df['tipo_movimiento'] == 'LIQUIDACION')]
for _, row in liq_31_df.iterrows():
    print(f"  ${abs(row['amount']):>8,.0f} -> {row['comercio']}")

print(f"\n⚠️  Se liquidaron ${liq_31:,.0f} pero solo había ${ing_30:,.0f} de ingreso previo.")
print(f"   Descuadre parcial: ${ing_30 - liq_31:,.0f}")

In [ ]:
print("=" * 80)
print("ANOMALÍA #4: DIFERENCIA SANTANDER → NOPAYMENTS")
print("=" * 80)

# Buscar el ingreso de Santander del 01/09
santander_sep01 = ingresos[
    (ingresos['date'] == '2022-09-01') & 
    (ingresos['banco'].str.contains('Santander', na=False))
]

# Buscar liquidación a NoPayments del 02/09 que corresponde
nopayments_sep02 = liquidaciones[
    (liquidaciones['date'] == '2022-09-02') & 
    (liquidaciones['comercio'] == 'NoPayments')
]

if len(santander_sep01) > 0:
    monto_ingreso = santander_sep01['amount'].values[0]
    print(f"\nIngreso Santander 01/09: ${monto_ingreso:,.0f}")

# Buscar la liquidación más cercana al monto
for _, row in nopayments_sep02.iterrows():
    if abs(row['amount']) > 3000000:  # Buscar el monto grande
        print(f"Liquidación NoPayments 02/09: ${abs(row['amount']):,.0f}")
        print(f"Diferencia: ${monto_ingreso + row['amount']:,.0f}")
        print(f"\n⚠️  La diferencia de $102 aparece como liquidación separada a 'Mirador San Juan'")

## 8. Timeline de Saldos

In [ ]:
# Crear timeline ordenado
df_timeline = df.sort_values('date').copy()
df_timeline['balance_acumulado'] = df_timeline['amount'].cumsum()

# Mostrar timeline
print("TIMELINE DE MOVIMIENTOS Y BALANCE ACUMULADO")
print("=" * 100)

for fecha in sorted(df_timeline['date'].unique()):
    movs_dia = df_timeline[df_timeline['date'] == fecha]
    balance_final = movs_dia['balance_acumulado'].iloc[-1]
    
    alerta = "⚠️" if balance_final < 0 else "  "
    
    print(f"\n{fecha.strftime('%Y-%m-%d')} ({fecha.strftime('%A')[:3]}) {alerta}")
    for _, row in movs_dia.iterrows():
        tipo = "+" if row['amount'] > 0 else "-"
        desc = row['description'][:50]
        print(f"  {tipo} ${abs(row['amount']):>12,.0f}  {desc}")
    print(f"  {'─' * 60}")
    print(f"  Balance: ${balance_final:>12,.0f}")

In [ ]:
# Visualización del balance acumulado
try:
    import matplotlib.pyplot as plt
    
    # Agrupar por fecha
    balance_diario = df_timeline.groupby('date')['balance_acumulado'].last()
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Gráfico de línea
    balance_diario.plot(ax=ax, marker='o', linewidth=2, markersize=6)
    
    # Línea de referencia en 0
    ax.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    
    # Sombrear área negativa
    ax.fill_between(balance_diario.index, balance_diario.values, 0, 
                    where=(balance_diario.values < 0), 
                    color='red', alpha=0.3, label='Balance negativo')
    
    ax.set_xlabel('Fecha')
    ax.set_ylabel('Balance (CLP)')
    ax.set_title('Evolución del Balance Acumulado - Débito Directo')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
except ImportError:
    print("Matplotlib no está instalado.")

## 9. Resumen de Anomalías

In [ ]:
print("\n" + "=" * 80)
print("RESUMEN DE ANOMALÍAS DETECTADAS")
print("=" * 80)

anomalias = [
    {
        'ID': 1,
        'Tipo': 'Descuadre financiero',
        'Descripción': 'Liquidaciones superan ingresos en $10,004',
        'Impacto': '$10,004 CLP',
        'Origen': 'Liquidaciones del 31/08 sin ingreso correspondiente'
    },
    {
        'ID': 2,
        'Tipo': 'Split incorrecto',
        'Descripción': 'Diferencia entre ingreso Santander y liquidación NoPayments',
        'Impacto': '$102 CLP',
        'Origen': 'Liquidación asignada incorrectamente a Mirador San Juan'
    },
    {
        'ID': 3,
        'Tipo': 'Calidad de datos',
        'Descripción': 'Comercios con nombres duplicados (diferencia de mayúsculas)',
        'Impacto': 'Data Quality',
        'Origen': 'Roots Housing y Smart Finances con variantes'
    },
    {
        'ID': 4,
        'Tipo': 'Movimientos atípicos',
        'Descripción': '12 transacciones con montos < $50',
        'Impacto': '12 movimientos',
        'Origen': 'Posibles pruebas de sistema o errores'
    }
]

df_anomalias = pd.DataFrame(anomalias)
print(df_anomalias.to_string(index=False))

## 10. Conclusiones

In [ ]:
print("\n" + "=" * 80)
print("CONCLUSIONES DEL ANÁLISIS")
print("=" * 80)

print("""
1. ¿SE CONDICE EL FUNCIONAMIENTO CON LOS MOVIMIENTOS?
   
   PARCIALMENTE. El flujo esperado (Ingreso día N → Liquidación día N+1 hábil)
   se cumple en la mayoría de los casos (8 de 11 fechas = 73%).
   Las 3 excepciones generan el descuadre detectado.

2. ANOMALÍAS ENCONTRADAS:
   
   a) Descuadre de $10,004 CLP - Se liquidó más de lo recibido
   b) Split incorrecto de $102 - Asignación errónea de liquidación  
   c) Comercios duplicados - Problema de normalización de datos
   d) Movimientos atípicos - 12 transacciones de montos muy pequeños

3. RECOMENDACIONES:
   
   a) Investigar urgente las liquidaciones del 31/08 sin ingreso asociado
   b) Revisar la diferencia de $102 entre Santander y NoPayments
   c) Normalizar nombres de comercios en el sistema
   d) Evaluar si movimientos < $50 son pruebas de sistema
""")

## 11. Exportar Resultados

In [ ]:
# Exportar tablas a CSV para uso posterior
import os

# Crear directorio de outputs si no existe
os.makedirs('outputs', exist_ok=True)

# Exportar análisis por banco
por_banco.to_csv('outputs/analisis_por_banco.csv')
print("Exportado: outputs/analisis_por_banco.csv")

# Exportar análisis por comercio
por_comercio.to_csv('outputs/analisis_por_comercio.csv')
print("Exportado: outputs/analisis_por_comercio.csv")

# Exportar verificación de flujo
df_verificacion.to_csv('outputs/verificacion_flujo.csv', index=False)
print("Exportado: outputs/verificacion_flujo.csv")

# Exportar anomalías
df_anomalias.to_csv('outputs/anomalias_detectadas.csv', index=False)
print("Exportado: outputs/anomalias_detectadas.csv")

# Exportar movimientos atípicos
montos_pequenos[['date', 'description', 'amount']].to_csv('outputs/movimientos_atipicos.csv', index=False)
print("Exportado: outputs/movimientos_atipicos.csv")

print("\nTodos los archivos exportados correctamente.")